In [1]:
import os
import numpy as np
import librosa
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
!pip install torch torchaudio transformers tqdm
from tqdm import tqdm
import torchaudio
from transformers import Wav2Vec2Processor, HubertForSequenceClassification
# Load the pretrained HuBERT model and processor
from transformers import AutoFeatureExtractor, HubertForSequenceClassification
!pip install torch torchvision torchaudio
import os
import numpy as np
import librosa
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape
import os
from tqdm import tqdm
import librosa
import torch
from sklearn.model_selection import train_test_split
from transformers import Wav2Vec2Processor, HubertModel, ViTConfig, ViTForImageClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score
import torch.nn as nn
import torch.optim as optim

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define the path to your dataset in Google Drive
dataset_path = '/content/drive/MyDrive/DATASET'

In [5]:
# Define subfolders
bonafide_path = os.path.join(dataset_path, 'Bonafide')
spoofed_tacotron_path = os.path.join(dataset_path, 'Spoofed_Tacotron')
spoofed_vits_tts_path = os.path.join(dataset_path, 'Spoofed_TTS')

# List all files in the dataset
def list_audio_files(folder_path):
    audio_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]
    return audio_files

bonafide_files = list_audio_files(bonafide_path)
spoofed_tacotron_files = list_audio_files(spoofed_tacotron_path)
spoofed_vits_tts_files = list_audio_files(spoofed_vits_tts_path)

print("Bonafied files:", bonafide_files)
print("Spoofed Tacotron files:", spoofed_tacotron_files)
print("Spoofed VitsTTS files:", spoofed_vits_tts_files)

Bonafied files: []
Spoofed Tacotron files: []
Spoofed VitsTTS files: []


In [6]:
# Initialize a dictionary to store file paths for each category
audio_files = {
    'Bonafide': [],
    'Spoofed_TTS': [],
    'Spoofed_Tacotron': []
}

# Iterate over the dataset folders (Bonafide, Spoofed_TTS, Spoofed_Tacotron)
for category in ['Bonafide', 'Spoofed_TTS', 'Spoofed_Tacotron']:
    category_path = os.path.join(dataset_path, category)

    for speaker_folder in os.listdir(category_path):
        speaker_path = os.path.join(category_path, speaker_folder)

        # Handle Bonafide folder with Part 1 and Part 2 subfolders
        if category == 'Bonafide':
            for part in ['Part 1', 'Part 2']:
                part_path = os.path.join(speaker_path, part)
                for file in tqdm(os.listdir(part_path), desc=f"Processing {category}/{speaker_folder}/{part}"):
                    file_path = os.path.join(part_path, file)
                    if file.endswith('.wav'):  # Check if it's a valid audio file
                        audio_files[category].append(file_path)

        # Handle Spoofed_TTS and Spoofed_Tacotron (assuming these don't have parts)
        else:
            for file in tqdm(os.listdir(speaker_path), desc=f"Processing {category}/{speaker_folder}"):
                file_path = os.path.join(speaker_path, file)
                if file.endswith('.wav'):  # Check if it's a valid audio file
                    audio_files[category].append(file_path)

# Check the loaded audio files
print("Bonafide files:", audio_files['Bonafide'])
print("Spoofed_TTS files:", audio_files['Spoofed_TTS'])
print("Spoofed_Tacotron files:", audio_files['Spoofed_Tacotron'])

Processing Spoofed_Tacotron/Speaker_01: 100%|██████████| 100/100 [00:00<00:00, 246000.23it/s]

Bonafide files: ['/content/drive/MyDrive/DATASET/Bonafide/Speaker_17/Part 1/10.wav', '/content/drive/MyDrive/DATASET/Bonafide/Speaker_17/Part 1/101.wav', '/content/drive/MyDrive/DATASET/Bonafide/Speaker_17/Part 1/102.wav', '/content/drive/MyDrive/DATASET/Bonafide/Speaker_17/Part 1/111.wav', '/content/drive/MyDrive/DATASET/Bonafide/Speaker_17/Part 1/306.wav', '/content/drive/MyDrive/DATASET/Bonafide/Speaker_17/Part 1/14.wav', '/content/drive/MyDrive/DATASET/Bonafide/Speaker_17/Part 1/105.wav', '/content/drive/MyDrive/DATASET/Bonafide/Speaker_17/Part 1/29.wav', '/content/drive/MyDrive/DATASET/Bonafide/Speaker_17/Part 1/273.wav', '/content/drive/MyDrive/DATASET/Bonafide/Speaker_17/Part 1/313.wav', '/content/drive/MyDrive/DATASET/Bonafide/Speaker_17/Part 1/114.wav', '/content/drive/MyDrive/DATASET/Bonafide/Speaker_17/Part 1/289.wav', '/content/drive/MyDrive/DATASET/Bonafide/Speaker_17/Part 1/129.wav', '/content/drive/MyDrive/DATASET/Bonafide/Speaker_17/Part 1/100.wav', '/content/drive/MyDr

In [7]:
# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [8]:
# Load the saved files
x_train = np.load("/content/drive/My Drive/x_train_embeddings.npy")
x_test = np.load("/content/drive/My Drive/x_test_embeddings.npy")
y_train = np.load("/content/drive/My Drive/y_train_labels.npy")
y_test = np.load("/content/drive/My Drive/y_test_labels.npy")

print("Embeddings and labels loaded from Google Drive.")

Embeddings and labels loaded from Google Drive.


In [9]:
print(f"x_train shape: {x_train.shape}")


x_train shape: (5447, 1, 32, 32)


In [10]:
import numpy as np
import math

# Function to pad embeddings to a square target size
def pad_to_square(embeddings, target_size):
    if embeddings.ndim != 2:
        raise ValueError("Input embeddings must be a 2D array")
    padding_length = target_size - embeddings.shape[1]
    if padding_length < 0:
        raise ValueError("Target size must be greater than or equal to the embeddings length")
    return np.pad(embeddings, ((0, 0), (0, padding_length)), mode='constant')

# Example data (replace with actual embeddings)
embedding_length = 1024  # Example embedding length
target_size = math.ceil(math.sqrt(embedding_length)) ** 2
print(f"Embedding length: {embedding_length}, Target size: {target_size}")

# Assume x_train and x_test are the embeddings arrays
# Example shapes: (num_samples, embedding_length)
x_train = np.random.rand(5447, embedding_length)
x_test = np.random.rand(1362, embedding_length)

x_train = pad_to_square(x_train, target_size)
x_test = pad_to_square(x_test, target_size)

height = width = int(math.sqrt(target_size))
x_train = x_train.reshape(x_train.shape[0], 1, height, width)
x_test = x_test.reshape(x_test.shape[0], 1, height, width)

print(f"x_train shape after padding and reshaping: {x_train.shape}")
print(f"x_test shape after padding and reshaping: {x_test.shape}")

Embedding length: 1024, Target size: 1024
x_train shape after padding and reshaping: (5447, 1, 32, 32)
x_test shape after padding and reshaping: (1362, 1, 32, 32)


In [11]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert to PyTorch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoaders
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [12]:
import torch
import torch.nn as nn
from torchvision.models import vit_b_16
from torch.optim import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [5]:
import numpy as np

# Load embeddings and labels from .npy files
x_train = np.load("/content/drive/My Drive/x_train_embeddings.npy")
x_test = np.load("/content/drive/My Drive/x_test_embeddings.npy")
y_train = np.load("/content/drive/My Drive/y_train_labels.npy")
y_test = np.load("/content/drive/My Drive/y_test_labels.npy")

# Ensure the embeddings are reshaped to (batch_size, 1, 32, 32)
x_train = x_train.reshape(x_train.shape[0], 1, 32, 32)
x_test = x_test.reshape(x_test.shape[0], 1, 32, 32)

print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")


x_train shape: (5447, 1, 32, 32)
x_test shape: (1362, 1, 32, 32)


In [7]:
# Prepare Dataset and DataLoader
train_dataset = CustomDataset(x_train, y_train)
test_dataset = CustomDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [12]:
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision import transforms
from transformers import ViTForImageClassification, ViTConfig
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Custom Dataset Class (No Resizing)
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        sample = self.x[idx]
        label = self.y[idx]
        return torch.tensor(sample, dtype=torch.float32), label

# Prepare Dataset and DataLoader
train_dataset = CustomDataset(x_train, y_train)
test_dataset = CustomDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Vision Transformer Configuration for 32x32 Input
config = ViTConfig(
    image_size=32,      # Input image size matches embeddings' reshaped size
    patch_size=4,       # Smaller patches for 32x32 input
    num_channels=1,     # Single channel (grayscale embeddings)
    num_labels=2,       # Binary classification (real/fake)
    hidden_size=512,    # Hidden dimension size
    num_attention_heads=8,  # Number of attention heads
    num_hidden_layers=6,    # Number of transformer blocks
    intermediate_size=2048, # Intermediate layer size
)

# Initialize Pre-Trained ViT Model for Image Classification
model = ViTForImageClassification(config)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training Function
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            # Move inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs).logits
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for inputs, labels in test_loader:
            # Move inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs).logits
            _, preds = torch.max(outputs, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    # Calculate Metrics
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1-Score: {f1:.4f}")

# Move the model to the device
model.to(device)

# Train and Evaluate the Model
train_model(model, train_loader, criterion, optimizer, num_epochs=10)
evaluate_model(model, test_loader)

Epoch [1/10], Loss: 0.7519
Epoch [2/10], Loss: 0.6681
Epoch [3/10], Loss: 0.5368
Epoch [4/10], Loss: 0.4163
Epoch [5/10], Loss: 0.3773
Epoch [6/10], Loss: 0.3124
Epoch [7/10], Loss: 0.2894
Epoch [8/10], Loss: 0.2877
Epoch [9/10], Loss: 0.2456
Epoch [10/10], Loss: 0.2236
Accuracy: 0.9097, Precision: 0.9548, Recall: 0.8559, F1-Score: 0.9026
